In [ ]:
import datetime
from db import DB
db = DB()

In [ ]:
import prep_download
import prep_parse

from_date = datetime.date(2019,12,9)
# prep_download.download_daily(db, from_date)
# prep_parse.parse_daily(db, from_date)

# prep_download.download_weekly(db, from_date)
# prep_parse.parse_weekly(db, from_date)

# prep_download.download_monthly(from_date)
# prep_parse.parse_monthly(db, from_date)

## https://mops.twse.com.tw/server-java/FileDownLoad?step=9&fileName=tifrs-2019Q3.zip&filePath=/home/html/nas/ifrs/2019/
# prep_download.download_quarterly(db, from_date)
# prep_parse.parse_quarterly(db, from_date)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('fivethirtyeight') # fivethirtyeight, fast
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # font to handle chinese
%matplotlib inline

In [ ]:
from analysis_fundamental import dashboard as fundamental
from analysis_technical import dashboard as technical
from analysis_chip import dashboard as chip

stock_id = '2317'
technical(stock_id, db)
chip(stock_id, db, 20)
fundamental(stock_id, db)

In [ ]:
import strategy
stock_info = db.get_stock_info()[['stock_name', 'industry']]
stock_data = strategy.prepare_stock_data(db)

In [ ]:
date = datetime.date(2019,11,8) # modify
df = strategy.get_stock_data_by_date(stock_info, stock_data, date)
df[ (df['EPS4季'] > 1) & (df['本益比'] < 10) & (df['眼光費'] < 0) ] # modify

In [ ]:
def backtest(stock_info, stock_data):
    for stock_id in stock_info.index:
        stock_name = stock_info.loc[stock_id, 'stock_name'] + ',' + stock_id
        df_daily = stock_data[stock_id]
        region = []
        get = False
        for i in range(len(df_daily.index)):
            if df_daily.iloc[i]['EPS4季'] > 1 and df_daily.iloc[i]['本益比'] < 10 and df_daily.iloc[i]['眼光費'] < 0:
                if not get:
                    region.append([df_daily.index[i]])
                    get = True
                continue
            if get:
                region[-1].append(df_daily.index[i - 1])
                get = False

        if len(region) > 0:
            if len(region[-1]) == 1:
                region[-1].append(df_daily.index[-1])
            ax = df_daily[['close']].plot(grid=True, title=stock_name, figsize=(10,4))
            for xmin, xmax in region:
                ax.axvspan(xmin, xmax, color='red', alpha=0.5)

backtest(stock_info, stock_data)